In [1]:
import pandas as pd
import heapq
import time
from collections import defaultdict
from google.colab import drive
!pip install openpyxl


# === Mount Google Drive ===
drive.mount('/content/drive', force_remount=True)

def read_graph(csv_path):
    df = pd.read_csv(csv_path)
    df.rename(columns={
        'Source Node  ID': 'source',
        'Target Node ID': 'target',
        'Edge Weight': 'weight'
    }, inplace=True)
    df['source'] = df['source'].astype(str)
    df['target'] = df['target'].astype(str)
    edges = list(df.itertuples(index=False, name=None))
    node_set = sorted(set(u for u, v, _ in edges).union(v for u, v, _ in edges))
    node_to_index = {node: i for i, node in enumerate(node_set)}
    index_to_node = {i: node for node, i in node_to_index.items()}
    edges_indexed = [(node_to_index[u], node_to_index[v], float(w)) for (u, v, w) in edges]
    return edges_indexed, node_to_index, index_to_node

Mounted at /content/drive


In [2]:
def load_initial_scores(csv_path, node_to_index):
    df = pd.read_csv(csv_path)
    df['Node ID'] = df['Node ID'].astype(str).str.strip()
    rank_map = {row['Node ID']: row['Order'] for _, row in df.iterrows()}

    scores = {}
    for node_str, idx in node_to_index.items():
        if node_str in rank_map:
            scores[idx] = int(rank_map[node_str])

    # Assign unique ranks to unranked nodes
    max_rank = max(scores.values(), default=0) + 1
    for node_str, idx in node_to_index.items():
        if idx not in scores:
            scores[idx] = max_rank
            max_rank += 1

    return scores


def compute_forward_weight(edges, scores):
    return sum(w for u, v, w in edges if scores[u] < scores[v])


In [3]:
def compute_all_gains_local(scores, u, v, out_edges, in_edges):
    idx_u, idx_v = scores[u], scores[v]
    if idx_u <= idx_v:
      raise ValueError(f"Invalid edge direction: edge ({u} → {v}) is already forward! Ranks: u={idx_u}, v={idx_v}")

    f2b_edges_mvubfv = set()

    def new_rank(node, strategy):
      if strategy == 'swap':
          if node == u:
              return idx_v
          elif node == v:
              return idx_u
      elif strategy == 'mvvafu':  # move v after u (v gets u's rank, others shift up)
          if node == v:
              return idx_u
          elif idx_v < scores[node] <=idx_u:
              return scores[node] - 1
      elif strategy == 'mvubfv':  # move u before v (u gets v's rank, others shift down)
          if node == u:
              return idx_v
          elif idx_v <= scores[node] < idx_u:
              return scores[node] + 1
      return scores[node]


    results = {
        'swap': {'gain': 0, 'f2b': 0, 'b2f': 0},
        'mvvafu': {'gain': 0, 'f2b': 0, 'b2f': 0},
        'mvubfv': {'gain': 0, 'f2b': 0, 'b2f': 0},
    }

    edges_checked = 0
    edge_uv_seen = False

    def process_edge(a, b, w):
        nonlocal edges_checked, edge_uv_seen,f2b_edges_mvubfv
        if (a == u and b == v) :
            if edge_uv_seen:
                return
            edge_uv_seen = True

        ra, rb = scores[a], scores[b]
        if not (
            (a in [u, v] and idx_v <= scores[b] <= idx_u) or
            (b in [u, v] and idx_v <= scores[a] <= idx_u)
        ):
            return

        edges_checked += 1
        old_forward = ra < rb

        for strategy in results:
            ra_new = new_rank(a, strategy)
            rb_new = new_rank(b, strategy)
            new_forward = ra_new < rb_new
            if (old_forward and (not new_forward)):
              results[strategy]['gain'] -= w
              results[strategy]['f2b'] += w
              if strategy == 'mvubfv':
                  f2b_edges_mvubfv.add((a, b, w))

            elif ((not old_forward) and new_forward):
                results[strategy]['gain'] += w
                results[strategy]['b2f'] += w
    lo, hi = idx_v, idx_u
    edges_to_check = set()

# Collect all relevant edges
    for x, w in out_edges[u]:
        if lo <= scores[x] <= hi:
            edges_to_check.add((u, x, w))
    for x, w in in_edges[u]:
        if lo <= scores[x] <= hi:
            edges_to_check.add((x, u, w))
    for x, w in out_edges[v]:
        if lo <= scores[x] <= hi:
            edges_to_check.add((v, x, w))
    for x, w in in_edges[v]:
        if lo <= scores[x] <= hi:
            edges_to_check.add((x, v, w))

    # Process them
    for a, b, w in edges_to_check:
        process_edge(a, b, w)



    return (
    results['swap']['gain'],
    results['mvvafu']['gain'],
    results['mvubfv']['gain'],
    edges_checked,
    results['swap']['f2b'], results['swap']['b2f'],
    results['mvvafu']['f2b'], results['mvvafu']['b2f'],
    results['mvubfv']['f2b'], results['mvubfv']['b2f'],
    f2b_edges_mvubfv

)
def compare_forward_weights(edges, sbef, saft):
    """
    Given edges and two score dictionaries (sbef and saft),
    returns:
    - sum of forward edge weights in sbef that are NOT forward in saft
    - sum of forward edge weights in saft that are NOT forward in sbef
    """
    sum_sbef_only = 0.0
    sum_saft_only = 0.0
    sum_both_forward = 0.0
    sum_both_backward = 0.0
    only_before_gobal=set()
    for u, v, w in edges:
        is_forward_sbef = sbef[u] < sbef[v]
        is_forward_saft = saft[u] < saft[v]

        if is_forward_sbef and not is_forward_saft:
            sum_sbef_only += w
            only_before_gobal.add((u,v,w))
        if is_forward_saft and not is_forward_sbef:
            sum_saft_only += w
        if is_forward_sbef and is_forward_saft:
            sum_both_forward += w
        if not is_forward_sbef and not is_forward_saft:
            sum_both_backward += w
    return sum_sbef_only, sum_saft_only, sum_both_forward, sum_both_backward, only_before_gobal
def all_scores_unique(scores):
    """
    Returns True if all nodes have unique scores, otherwise False.
    """
    values = list(scores.values())
    return len(values) == len(set(values))

In [4]:
def compute_prefix_sums(sorted_neighbors, target_node, edges_dict, reverse=False):
    """
    Compute prefix (or suffix if reverse=True) sums of outgoing and ingoing edges
    between target_node and each neighbor in sorted_neighbors.
    """
    ingoing = []
    outgoing = []
    in_sum = out_sum = 0

    for idx in range(len(sorted_neighbors)):
        neighbor = sorted_neighbors[idx] if not reverse else sorted_neighbors[-1 - idx]
        key_out = (target_node, neighbor)
        key_in = (neighbor, target_node)

        if key_out in edges_dict:
            out_sum += edges_dict[key_out]
        if key_in in edges_dict:
            in_sum += edges_dict[key_in]

        if not reverse:
            outgoing.append(out_sum)
            ingoing.append(in_sum)
        else:
            outgoing.insert(0, out_sum)
            ingoing.insert(0, in_sum)

    return outgoing, ingoing

def compute_and_print_delta_gain(u, v, w_uv, w_vu, ingoing_v, outgoing_v, ingoing_u, outgoing_u):
    """
    Prints the detailed delta gain computation for moving edge (u → v) into forward direction.
    """
    ingoing_v_i = ingoing_v[v]
    outgoing_v_i = outgoing_v[v]
    ingoing_u_j = ingoing_u[u]
    outgoing_u_j = outgoing_u[u]

    delta = w_uv - w_vu + ingoing_v_i - outgoing_v_i - ingoing_u_j + outgoing_u_j

    print(f"--- Gain Computation for Edge {u} → {v} ---")
    print(f"w_uv       = {w_uv}")
    print(f"w_vu       = {w_vu}")
    print(f"ingoing_v  = {ingoing_v_i}")
    print(f"outgoing_v = {outgoing_v_i}")
    print(f"ingoing_u  = {ingoing_u_j}")
    print(f"outgoing_u = {outgoing_u_j}")
    print(f"Delta gain = {delta:.2f}")
    print("=====================================\n")

    return delta




from bisect import bisect_right

def apply_new_strategy(scores, u, v, between, out_edges, in_edges, edges_dict, debug=False):
    between= sorted(between, key=lambda x: scores[x])

    frozen_scores = scores.copy()  # ✅ Preserve original scores for "before" debug views
    ranks_before = (scores[u], scores[v])
    n = len(scores)

    # Assert all between nodes have valid scores
    for node in between:
        assert scores[v] < scores[node] < scores[u], f"Invalid score for node {node} in between"

    # Define neighbors of v and u within 'between'
    v_neighbors = [node for node in between if (node, v) in edges_dict or (v, node) in edges_dict]
    u_neighbors = [node for node in between if (node, u) in edges_dict or (u, node) in edges_dict]

    # Sort neighbors by frozen scores (before any modification)
    v_neighbors_sorted_before = sorted(v_neighbors, key=lambda x: frozen_scores[x])

    u_neighbors_sorted_before = sorted(u_neighbors, key=lambda x: frozen_scores[x])
    u_scores = [frozen_scores[node] for node in u_neighbors_sorted_before]
    assert all(u_scores[i] <= u_scores[i + 1] for i in range(len(u_scores) - 1)), "u_scores is not sorted!"

    # Precompute prefix sums for v_neighbors
    outgoing_v = [0] * (len(v_neighbors_sorted_before) + 1)
    ingoing_v = [0] * (len(v_neighbors_sorted_before) + 1)
    for i, node in enumerate(v_neighbors_sorted_before):
        outgoing_v[i + 1] = outgoing_v[i] + edges_dict.get((v, node), 0)
        ingoing_v[i + 1] = ingoing_v[i] + edges_dict.get((node, v), 0)

    # Precompute suffix sums for u_neighbors
    outgoing_u = [0] * (len(u_neighbors_sorted_before) + 1)
    ingoing_u = [0] * (len(u_neighbors_sorted_before) + 1)
    for i in reversed(range(len(u_neighbors_sorted_before))):
        outgoing_u[i] = outgoing_u[i + 1] + edges_dict.get((u, u_neighbors_sorted_before[i]), 0)
        ingoing_u[i] = ingoing_u[i + 1] + edges_dict.get((u_neighbors_sorted_before[i], u), 0)

    # Find best cut
    best_delta = float('-inf')
    best_y = -1
    best_j = -1
    w_uv = edges_dict.get((u, v), 0.0)
    w_vu = edges_dict.get((v, u), 0.0)

    for i, nv in enumerate(v_neighbors_sorted_before):
        nv_score = frozen_scores[nv]
        j = bisect_right(u_scores, nv_score)
        if j < len(u_scores):
            delta = w_uv - w_vu + ingoing_v[i+1] - outgoing_v[i+1] - ingoing_u[j] + outgoing_u[j]
            if delta > best_delta:
                best_delta = delta
                best_y = i
                best_j = j

    if best_delta <= 0:
        return False, 0.0, ranks_before, (scores[u], scores[v]), {}

    # Apply reordering
    # Apply reordering
    split_point = between.index(v_neighbors_sorted_before[best_y])
    left = between[:split_point + 1]
    right = between[split_point + 1:]
    new_order = left + [u, v] + right
    original_block = [v] + between + [u]
    base_score = scores[v]

    # Backup scores
    old_scores_snapshot = {node: scores[node] for node in original_block}
    split_point_node = v_neighbors_sorted_before[best_y]
    old_split_score = old_scores_snapshot[split_point_node]

    for i, node in enumerate(new_order):
        new_score = base_score + i
        old_score = scores[node]
        scores[node] = new_score

        if node not in {u, v}:
            delta = abs(new_score - old_score)
            if delta > 1:
                print("\n❌ Rank change violation for a non-u/v node!")
                print(f"🔸 Violating node: {node}")
                print(f"🔹 Old Score based on scores: {old_score}")
                print(f"🔹 Old Score based on old_scores_snapshot: {old_scores_snapshot[node]}")
                print(f"🔹 New Score: {new_score}")
                print(f"🔹 Delta: {delta}")
                print(f"📌 Split point node: {split_point_node}")
                print(f"    ↳ Old rank = {old_split_score}")
                print(f"    ↳ New rank = {scores[split_point_node]}")
                print(f"📏 Length of LEFT block = {len(left)}")
                print(f"📏 Length of RIGHT block = {len(right)}")
                print(f"u = {u}, old rank = {old_scores_snapshot[u]}, new rank = {scores[u]}")
                print(f"v = {v}, old rank = {old_scores_snapshot[v]}, new rank = {scores[v]}")
                print(f"base_score = {base_score}")

                print("\n📜 Previous order with scores:")
                for n in original_block:
                    print(f"{n}: {old_scores_snapshot[n]}")

                print("\n📜 New order with scores:")
                for n in new_order:
                    print(f"{n}: {scores[n]}")

                raise ValueError(f"Node {node} changed score by {delta}, which exceeds limit.")




    ranks_after = (scores[u], scores[v])

    # Re-sort neighbors based on updated scores
    v_neighbors_sorted_after = sorted(v_neighbors, key=lambda x: scores[x])
    #v_neighbors_sorted_after=[]
    u_neighbors_sorted_after = sorted(u_neighbors, key=lambda x: scores[x])
    #u_neighbors_sorted_after = []
    # Debug information
    debug_info = {
        "v_neighbors_before": [(node, frozen_scores[node]) for node in v_neighbors_sorted_before],
        "u_neighbors_before": [(node, frozen_scores[node]) for node in u_neighbors_sorted_before],
        "v_neighbors_after": [(node, scores[node]) for node in v_neighbors_sorted_after],
        "u_neighbors_after": [(node, scores[node]) for node in u_neighbors_sorted_after],
        "ingoing_v": ingoing_v,
        "outgoing_v": outgoing_v,
        "ingoing_u": ingoing_u,
        "outgoing_u": outgoing_u,
        "best_y": best_y,
        "best_j": best_j,
        "best_delta": best_delta,
        "gain_info": {
            "w_uv": w_uv,
            "w_vu": w_vu,
            "ing_v_i": {
                "total": ingoing_v[best_y + 1],
                "details": [
                    {"node": nv, "score": frozen_scores[nv], "weight": edges_dict.get((nv, v), 0)}
                    for nv in v_neighbors_sorted_before[:best_y + 1]
                    if (nv, v) in edges_dict
                ]
            },
            "out_v_i": {
                "total": outgoing_v[best_y + 1],
                "details": [
                    {"node": nv, "score": frozen_scores[nv], "weight": edges_dict.get((v, nv), 0)}
                    for nv in v_neighbors_sorted_before[:best_y + 1]
                    if (v, nv) in edges_dict
                ]
            },
            "ing_u_j": {
                "total": ingoing_u[best_j],
                "details": [
                    {"node": nu, "score": frozen_scores[nu], "weight": edges_dict.get((nu, u), 0)}
                    for nu in u_neighbors_sorted_before[best_j:]
                    if (nu, u) in edges_dict
                ]
            },
            "out_u_j": {
                "total": outgoing_u[best_j],
                "details": [
                    {"node": nu, "score": frozen_scores[nu], "weight": edges_dict.get((u, nu), 0)}
                    for nu in u_neighbors_sorted_before[best_j:]
                    if (u, nu) in edges_dict
                ]
            },
            "delta": best_delta,
            "best_i": best_y,
            "best_j": best_j
        }
    }


    return True, best_delta, ranks_before, ranks_after, debug_info









In [5]:
import pandas as pd
import heapq
from collections import defaultdict

def refine_ranking_with_extended_strategy(csv_path, initial_ranking_path, output_excel):
    print("🚀 Starting refinement with extended strategy...")

    edges, node_to_index, index_to_node = read_graph(csv_path)
    n = len(node_to_index)
    scores = load_initial_scores(initial_ranking_path, node_to_index)

    total_weight = sum(w for _, _, w in edges)
    forward_weight = compute_forward_weight(edges, scores)

    print(f"✅ Initial forward weight / total = {forward_weight:.2f} / {total_weight:.2f} = {forward_weight / total_weight:.4f}")

    max_heap = []
    out_edges, in_edges = defaultdict(list), defaultdict(list)
    seen = set()
    edges_dict = {}

    for u, v, w in edges:
        out_edges[u].append((v, w))
        in_edges[v].append((u, w))
        edges_dict[(u, v)] = w
        if scores[u] > scores[v]:
            heapq.heappush(max_heap, (-w, u, v))
            seen.add((u, v))

    accepted = rejected = extended_accepted = total_checked = 0
    latest_extended_debug = {}
    last_debug_info = None

    while max_heap:
        total_checked += 1
        neg_w, u, v = heapq.heappop(max_heap)

        if scores[u] < scores[v]:
            continue

        idx_u, idx_v = scores[u], scores[v]
        if idx_u <= idx_v:
            raise ValueError(f"Logic error: backward edge ({u}→{v}) has idx_u={idx_u} <= idx_v={idx_v}")

        between = [node for node, rank in scores.items() if idx_v < rank < idx_u]
        scores_before = scores.copy()

        success, delta, ranks_before, ranks_after, debug_info = apply_new_strategy(
            scores, u, v, between, out_edges, in_edges, edges_dict, debug=True
        )

        improvement = False
        new_order = sorted([v, u] + between, key=lambda x: scores[x]) if success else []
     #   success=False
        if success:
            latest_extended_debug = debug_info
            last_debug_info = debug_info
            extended_accepted += 1
            forward_weight += delta
            improvement = True
            print("\n✨ Extended strategy applied")
            print(f"Edge: ({u} → {v})")
            print(f"Ranks before: u={ranks_before[0]}, v={ranks_before[1]}")
            print(f"Ranks after: u={ranks_after[0]}, v={ranks_after[1]}")
            print(f"Delta gain: {delta:.2f}, updated forward weight: {forward_weight:.2f}")
            print(f"Nodes between: {between}")

            if debug_info.get("gain_info"):
              gi = debug_info["gain_info"]
              print("📌 Gain formula breakdown (i, j yielding max delta):")
              print(f"w_uv         = {gi['w_uv']}")
              print(f"w_vu         = {gi['w_vu']}")
              print(f"ingoing_v[i] = {gi['ing_v_i']['total']}")
              print(f"outgoing_v[i]= {gi['out_v_i']['total']}")
              print(f"ingoing_u[j] = {gi['ing_u_j']['total']}")
              print(f"outgoing_u[j]= {gi['out_u_j']['total']}")
              print(f"delta gain   = {gi['delta']:.2f}")
              print(f"best_y (v) index = {debug_info['best_y']}")
              print(f"best_j (u) index = {debug_info['best_j']}")

              print("\n📊 Neighbors of v (before scores updated):")
              for node, score in debug_info["v_neighbors_before"]:
                  print(f"  Node: {node}, Score: {score}")

              print("\n📊 Neighbors of u (before scores updated):")
              for node, score in debug_info["u_neighbors_before"]:
                  print(f"  Node: {node}, Score: {score}")

              print("\n📊 Neighbors of v (after scores updated):")
              for node, score in debug_info["v_neighbors_after"]:
                  print(f"  Node: {node}, Score: {score}")

              print("\n📊 Neighbors of u (after scores updated):")
              for node, score in debug_info["u_neighbors_after"]:
                  print(f"  Node: {node}, Score: {score}")

              print("\n📈 Ingoing V Prefix Sum Array:", debug_info["ingoing_v"])
              print("📈 Outgoing V Prefix Sum Array:", debug_info["outgoing_v"])
              print("📈 Ingoing U Suffix Sum Array:", debug_info["ingoing_u"])
              print("📈 Outgoing U Suffix Sum Array:", debug_info["outgoing_u"])



        else:
            result = compute_all_gains_local(scores, u, v, out_edges, in_edges)
            gain1, gain2, gain3, checked, *_ = result

            best_gain = max(gain1, gain2, gain3)
            if best_gain <= 0:
                rejected += 1
                continue

            if best_gain == gain1:
                scores[u], scores[v] = idx_v, idx_u
            elif best_gain == gain2:
                for k, r in scores.items():
                    if idx_v < r <= idx_u:
                        scores[k] -= 1
                scores[v] = idx_u
            elif best_gain == gain3:
                for k, r in scores.items():
                    if idx_v <= r < idx_u:
                        scores[k] += 1
                scores[u] = idx_v

            last_debug_info = {
                "gain_info": {
                    "strategy": "greedy",
                    "delta": best_gain,
                    "u": u,
                    "v": v
                }
            }

            forward_weight += best_gain
            accepted += 1
            improvement = True
            new_order = sorted([v, u] + between, key=lambda x: scores[x])

        if improvement:
          real_fw = compute_forward_weight(edges, scores)
          if abs(real_fw - forward_weight) > 1e-4:
              print("\n❌ ERROR: Forward weight mismatch!")
              print(f"Expected forward weight: {forward_weight}, actual: {real_fw}")

              if last_debug_info:
                  gain_info = last_debug_info.get("gain_info", None)
                  if gain_info:
                      print("📌 Gain formula breakdown (most recent move):")
                      for k, v in gain_info.items():
                          print(f"{k} = {v}")

              raise ValueError("Forward weight mismatch. Terminating.")

          # Load existing output ranking to compare forward weight
          import os

          if not os.path.exists(output_excel):
              print("⚠️ Existing output file not found. Treating existing FW as 0.")
              existing_fw = 0
              existing_order = {}
          else:
              df_existing = pd.read_csv(output_excel)

# Check required columns
              if "Node ID" not in df_existing.columns or "Order" not in df_existing.columns:
                  print("⚠️ Existing file is malformed or missing columns. Skipping comparison.")
                  existing_fw = 0
                  existing_order = {}
              else:
                  df_existing['Node ID'] = df_existing['Node ID'].astype(str).str.strip()

                  # Map node string IDs to order from file
                  rank_map = dict(zip(df_existing['Node ID'], df_existing['Order']))

                  # Build internal score map using index
                  existing_scores = {}
                  for node_str, idx in node_to_index.items():
                      if node_str in rank_map:
                          existing_scores[idx] = int(rank_map[node_str])

                  # Assign unique ranks to unranked nodes
                  max_rank = max(existing_scores.values(), default=0) + 1
                  for node_str, idx in node_to_index.items():
                      if idx not in existing_scores:
                          existing_scores[idx] = max_rank
                          max_rank += 1

                  # Define existing_order using string keys for restoration
                  existing_order = {str(index_to_node[idx]): rank for idx, rank in existing_scores.items()}

                  # Compare forward weights
                  existing_fw = compute_forward_weight(edges, existing_scores)
                  print(f"\n🧮 Existing FW = {existing_fw:.2f}, New FW = {real_fw:.2f}")

                  if real_fw > existing_fw:
                      print("✅ Improvement confirmed — writing new ranking.")
                      ranked = [(str(index_to_node[v]), scores[v]) for v in range(n)]
                      ranked.sort(key=lambda x: x[1])
                      pd.DataFrame(ranked, columns=["Node ID", "Order"]).to_csv(output_excel, index=False)
                      print(f"💾 Intermediate ranking saved to {output_excel}")
                  elif existing_fw > real_fw:
                      forward_weight=existing_fw
                      print("↩️ Existing ranking is better — restoring it.")
                      for node_str, order in existing_order.items():
                          idx = node_to_index.get(node_str)
                          if idx is not None:
                              scores[idx] = order
                  else:
                      print("⏭️ Equal forward weight — skipping write and keeping current ranking.")




        for node in (u, v):
            for nbr, w in out_edges[node]:
                if scores[node] > scores[nbr] and (node, nbr) not in seen:
                    heapq.heappush(max_heap, (-w, node, nbr))
                    seen.add((node, nbr))
            for nbr, w in in_edges[node]:
                if scores[nbr] > scores[node] and (nbr, node) not in seen:
                    heapq.heappush(max_heap, (-w, nbr, node))
                    seen.add((nbr, node))

    ranked = [(str(index_to_node[v]), scores[v]) for v in range(n)]
    ranked.sort(key=lambda x: x[1])
    pd.DataFrame(ranked, columns=["Node ID", "Order"]).to_csv(output_excel, index=False)

    print("\n✅ Refinement complete.")
    print(f"Total accepted moves: {accepted}, extended strategy moves: {extended_accepted}, rejected: {rejected}")
    print(f"Final forward weight: {forward_weight:.2f} / {total_weight:.2f} ({forward_weight / total_weight:.4f})")
    print(f"Final ranking saved to {output_excel}")






In [ ]:
refine_ranking_with_extended_strategy(
    csv_path="/content/drive/MyDrive/connectome_graph.csv",
    initial_ranking_path="/content/drive/MyDrive/35461047-Soroush-Ioannis-advancedimprove.csv",
    output_excel="/content/drive/MyDrive/35461047-Soroush-Ioannis-advancedimprove.csv"
)

🚀 Starting refinement with extended strategy...
✅ Initial forward weight / total = 35462921.00 / 41912141.00 = 0.8461
